
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>





# Propagating Changes with CDF Lab

We'll be using [Change Data Feed](https://docs.databricks.com/en/delta/delta-change-data-feed.html) to propagate changes to many tables from a single source.

For this lab, we'll work with the fitness tracker datasets to propagate changes through a Lakehouse with Delta Lake Change Data Feed (CDF).

Because the **user_lookup** table links identifying information between different pipelines, we'll make this the point where changes propagate from.


## Objectives
By the end of this lab, you should be able to:
- Enable Change Data Feed on a particular table
- Read CDF output with Spark SQL or PySpark
- Refactor ELT code to process CDF output

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

    - In the drop-down, select **More**.

    - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## A. Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to your unique catalog name and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG your-catalog;
USE SCHEMA your-catalog.pii_data;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
%run ./Includes/Classroom-Setup-1.4L

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


Creating the users_bin table within your Catalog's cdf_lab schema.
Creating the user_lookup table within your Catalog's cdf_lab schema.
Creating the users table within your Catalog's cdf_lab schema.


Your Course Catalog Name:,
Your Schema:,


Check your default catalog and schema. Notice the lab uses a new schema named **cdf_lab**.

In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
labuser10792347_1751575757,cdf_lab



## B. Show available tables

We'll focus on the **user_lookup** table with the intent to delete a user, check the CDF changes and then propagate such changes into the tables **users** and **users_bin**.

Run the `SHOW TABLES` command and confirm the following tables are available:

- **user_lookup**: Table containing user lookup information.
- **users**: Table containing detailed user information.
- **users_bin**: Table containing user bin assignments.

In [0]:
SHOW TABLES;

database,tableName,isTemporary
cdf_lab,user_lookup,false
cdf_lab,users,false
cdf_lab,users_bin,false



## C. Review the user_lookup table data

Run the following query to view the **user_lookup** table. Notice that the table contains an **alt_id** column, as well as **device_id**, **mac_address**, and **user_id**.


In [0]:
SELECT *
FROM user_lookup
LIMIT 10;

alt_id,device_id,mac_address,user_id
745c35775cfdf69e9a190c26a173285712c1929c5faa5087ce2855e9043b33f4,190960,14:cd:d6:db:70:f6,11745
a610459add89c3c6a1de87bf8beaa4aee317564837e8d0247852004197e7d8cc,141687,ae:ec:f6:48:ca:f7,12140
c3274e338a1da85e0748f6c9c92247e176044770f4e39637a4c4d4b348191fcf,114131,57:24:ac:8c:75:ea,12227
e6a6387ffbb75e14eb39b138be1e7e9b8a6687e66c36865bf6e234d6688dd517,118440,4c:c5:9f:cb:13:bd,12474
898fa1b7f7057b1b17bca1342b4341b7d84e228d6d9bb83a7a770c4d487ebc4b,109290,36:1f:d9:d3:e8:0d,13559
e83442f287db35be0374a66e2253ab5c119417b0e559554291ac2af8f7f5809f,143442,dd:96:be:e9:1e:f4,13937
71324d786c7f10bd56edf127946a934016c65ff515c47f4d6ea5d233d5cae66a,172965,dd:45:d2:37:a8:0e,14232
a6f9678bccb4d7e7bc0ba5afbe9ba83c154889506173f51c1d55bc19c115bec6,102558,df:f9:dc:5e:e2:a8,14508
8bdbabab0376e9bc29748fd75ba8b669e9947dbcb0ee492a32a4d5c1b604321f,175406,1d:69:69:75:d0:aa,14633
54c9a8be55ad6b77cc376ef3638fc4af1ae1961685edd4cd986a5376c46bc67f,177966,de:c0:cd:a7:71:f4,15149



## D. Enable CDF for the user_lookup table

Enable change data feed on the **user_lookup** table.

**HINT:** Use the `ALTER TABLE` and `SET TBLPROPERTIES` to activate CDF. View the [Enable change data feed](https://docs.databricks.com/en/delta/delta-change-data-feed.html#enable-change-data-feed) documentation for more information.


In [0]:
--<FILL IN>


## E. Confirm CDF is enabled on user_lookup table

**HINT**: Use the `DESCRIBE TABLE EXTENDED` statement to view information about the **user_lookup** table. Scroll to the bottom of the results and view the *Table Properties* row. Confirm that this table has `delta.enableChangeDataFeed` activated.



In [0]:
--<FILL IN>


## F. View user_lookup table History

Enabling CDF generates a new entry in the table's history.


View the history of the **user_lookup** table. Confirm the entry is in the history of the **user_lookup** table as version `1` and that the **operation** column has updated the *SET TBLPROPERTIES*. 


In [0]:
--<FILL IN>


## G. Delete a record from user_lookup table



### G1. Query and pick a record from the User Lookup table

Run the query below and pick a **user_id** value to use in the next section. Notice that there are 100 records in the **user_lookup** table.

In [0]:
SELECT * 
FROM user_lookup

alt_id,device_id,mac_address,user_id
745c35775cfdf69e9a190c26a173285712c1929c5faa5087ce2855e9043b33f4,190960,14:cd:d6:db:70:f6,11745
a610459add89c3c6a1de87bf8beaa4aee317564837e8d0247852004197e7d8cc,141687,ae:ec:f6:48:ca:f7,12140
c3274e338a1da85e0748f6c9c92247e176044770f4e39637a4c4d4b348191fcf,114131,57:24:ac:8c:75:ea,12227
e6a6387ffbb75e14eb39b138be1e7e9b8a6687e66c36865bf6e234d6688dd517,118440,4c:c5:9f:cb:13:bd,12474
898fa1b7f7057b1b17bca1342b4341b7d84e228d6d9bb83a7a770c4d487ebc4b,109290,36:1f:d9:d3:e8:0d,13559
e83442f287db35be0374a66e2253ab5c119417b0e559554291ac2af8f7f5809f,143442,dd:96:be:e9:1e:f4,13937
71324d786c7f10bd56edf127946a934016c65ff515c47f4d6ea5d233d5cae66a,172965,dd:45:d2:37:a8:0e,14232
a6f9678bccb4d7e7bc0ba5afbe9ba83c154889506173f51c1d55bc19c115bec6,102558,df:f9:dc:5e:e2:a8,14508
8bdbabab0376e9bc29748fd75ba8b669e9947dbcb0ee492a32a4d5c1b604321f,175406,1d:69:69:75:d0:aa,14633
54c9a8be55ad6b77cc376ef3638fc4af1ae1961685edd4cd986a5376c46bc67f,177966,de:c0:cd:a7:71:f4,15149


### G2. Delete a record from User Lookup table

To delete record from the table pick a value from the **user_id** column from above to delete. Then complete the following:

- Place the picked **user_id** value after the `DEFAULT` keyword in the `DECLARE OR REPLACE VARIABLE` statement below. This will save the value into a variable named **user_id** that will be used across this lab.
- Use `DELETE` statement with column_name of table
- Run the cell below to proceed with the record's deletion from the table. 
- Confirm a row was deleted.

In [0]:
<FILL IN>

-- Define a variable for user_id to delete
DECLARE OR REPLACE VARIABLE user_id INT DEFAULT <FILL IN>;       -- Place your user_id here

-- Use the variable in the DELETE statement
DELETE FROM user_lookup 
WHERE user_id = session.user_id;

org.apache.spark.sql.catalyst.parser.ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near '<'. SQLSTATE: 42601 (line 1, pos 0)

== SQL ==
<FILL IN>
^^^

-- Define a variable for user_id to delete
DECLARE OR REPLACE VARIABLE user_id INT DEFAULT <FILL IN>

	at org.apache.spark.sql.catalyst.parser.ParseException.withCommand(parsers.scala:429)
	at org.apache.spark.sql.catalyst.parser.AbstractParser.parse(parsers.scala:117)
	at org.apache.spark.sql.execution.SparkSqlParser.parse(SparkSqlParser.scala:140)
	at org.apache.spark.sql.catalyst.parser.AbstractSqlParser.parsePlan(AbstractSqlParser.scala:106)
	at com.databricks.sql.parser.DatabricksSqlParser.$anonfun$parsePlan$1(DatabricksSqlParser.scala:80)
	at com.databricks.sql.parser.DatabricksSqlParser.parse(DatabricksSqlParser.scala:101)
	at com.databricks.sql.parser.DatabricksSqlParser.parsePlan(DatabricksSqlParser.scala:77)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$6(SparkSession.scala:978)
	at org.apache.spark.sql.catalyst

### G3. Confirm the record is deleted from the User Lookup table

Run the query below to confirm the selected **user_id** was deleted from the **user_lookup** table. Confirm no rows are returned.

In [0]:
SELECT * 
FROM user_lookup
WHERE user_id=session.user_id;

org.apache.spark.sql.catalyst.ExtendedAnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `session`.`user_id` cannot be resolved. Did you mean one of the following? [`user_lookup`.`user_id`, `user_lookup`.`alt_id`, `user_lookup`.`device_id`, `user_lookup`.`mac_address`]. SQLSTATE: 42703; line 3 pos 14;
'Project [*]
+- 'Filter (user_id#30419L = 'session.user_id)
   +- SubqueryAlias labuser10792347_1751575757.cdf_lab.user_lookup
      +- Relation labuser10792347_1751575757.cdf_lab.user_lookup[alt_id#30416,device_id#30417L,mac_address#30418,user_id#30419L] parquet

	at org.apache.spark.sql.errors.QueryCompilationErrors$.unresolvedAttributeError(QueryCompilationErrors.scala:454)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.org$apache$spark$sql$catalyst$analysis$CheckAnalysis$$failUnresolvedAttribute(CheckAnalysis.scala:171)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis0$8(CheckAnalysis.scal

Run the query below and notice that the table now holds *99* records.

In [0]:
SELECT count(*)
FROM user_lookup;

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

## H. Review user_lookup table History

Now that our record is deleted, a new entry in the **user_lookup** table history is added corresponding to this event. Let's get a view to this table's history. 

  Run the query and confirm version **2** and that the **operation** column is *DELETE*.

In [0]:
DESCRIBE HISTORY user_lookup;

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr


## I. Read the user_lookup CDF output (starting from version 1)

This section we'll proceed to read CDF data from a the **user_lookup** table, well apply two approaches:
- Using SQL via `table_changes` function
- Using Python

### I1. Using SQL to View Row Level Changes

To read the CDF data:

- Using [table_changes](https://docs.databricks.com/en/sql/language-manual/functions/table_changes.html) function
- Select all changes from version **1** of the **user_lookup** table

Confirm that one row is returned in the output. The row should be the row you deleted in the earlier step.

**HINT:** [Use Delta Lake change data feed on Databricks](https://docs.databricks.com/en/delta/delta-change-data-feed.html#use-delta-lake-change-data-feed-on-databricks) documentation.

In [0]:
<FILL IN>

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

### I2. (OPTIONAL) Using Python to View Row Level Changes 

To read the CDF data:
- Read the **user_lookup** table
- Configure the stream to enable reading change data with the `readChangeData` option
- Configure the stream to start reading from version **1** with `startingVersion` option

Confirm that one row is returned in the output. The row should be the row you deleted in the earlier step.

**HINT:** [Use Delta Lake change data feed on Databricks](https://docs.databricks.com/en/delta/delta-change-data-feed.html#use-delta-lake-change-data-feed-on-databricks) documentation.

In [0]:
<FILL IN>

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

## J. Propagate Deletes To Multiple Tables

### J1. Create a Temporary View of the user_lookup Table CDF Changes

Let's create a temporary view to hold the CDF changes of **user_lookup** table.

**HINTS**:
- Use `CREATE OR REPLACE TEMPORARY VIEW` name it **user_lookup_deletes_vw**
- Use `table_changes` and starting version of **2**
- Select all records in view where **_change_type** is *delete*
- Then display the results of the view. 
- Confirm one row is returned.

In [0]:
<FILL-IN>

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

### J2. Merge the Temporary View into **Users** Table

Merge using **user_lookup_deletes_vw** as source into **users** table and **DELETE** when **alt_id** gets matched.

The `MERGE` operation should return a value of *1* for **num_affected_rows** and **num_deleted_rows**.

**HINTS**:
- Use the [MERGE INTO](https://docs.databricks.com/en/sql/language-manual/delta-merge-into.html) Syntax:
```
MERGE [ WITH SCHEMA EVOLUTION ] INTO target_table_name [target_alias]
   USING source_table_reference [source_alias]
   ON merge_condition
   { WHEN MATCHED [ AND matched_condition ] THEN matched_action |
     WHEN NOT MATCHED [BY TARGET] [ AND not_matched_condition ] THEN not_matched_action |
     WHEN NOT MATCHED BY SOURCE [ AND not_matched_by_source_condition ] THEN not_matched_by_source_action } [...]

matched_action
 { DELETE |
   UPDATE SET * |
   UPDATE SET { column = { expr | DEFAULT } } [, ...] }

not_matched_action
 { INSERT * |
   INSERT (column1 [, ...] ) VALUES ( expr | DEFAULT ] [, ...] )

not_matched_by_source_action
 { DELETE |
   UPDATE SET { column = { expr | DEFAULT } } [, ...] }

```

In [0]:
<FILL_IN>

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

Run the query below to check the record count in the **users** table, notice this reflects record deleted propagated via the `MERGE_INTO` executed in the cell above.

In [0]:
SELECT count(*)
FROM users

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

### J3. Merge the Temporary View into the users_bin Table

Similarly, merge using **user_lookup_deletes_vw** as source into **user_bins** table and `DELETE` when **user_id** gets matched.

The `MERGE` operation should return a value of *1* for **num_affected_rows** and **num_deleted_rows**.

In [0]:
<FILL_IN>

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr

Run the query below to check for the deleted record in the **users_bin** table by filtering and using the selected **user_id** saved in a variable. 

Notice there are no results confirming the delete as correctly propagated via the `MERGE INTO`.

In [0]:
SELECT *
FROM users_bin
WHERE user_id = session.user_id

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:464)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:571)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:656)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContextTracing.scala:48)
	at com.databricks.logging.AttributionContext$.$anonfun$withValue$1(AttributionContext.scala:276)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(Attr


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>
